<a href="https://colab.research.google.com/github/rahiakela/transformers-research-and-practice/blob/main/transformers-for-natural-language-processing/9-semantic-role-labeling-with-bert-based-transformers/semantic_role_labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Semantic Role Labeling(SRL)


SRL labels the semantic role as the role a word or group of words plays in a sentence and the relationship established with the predicate.

A semantic role is the role a noun or noun phrase plays in relation to the main verb in a sentence.

The predicate describes something about the subject or agent. The predicate could be anything
that provides information on the features or actions of a subject.

The noun or noun phrases that revolve around the predicate are arguments or argument terms.




We will run our SRL experiments using the BERT SRL. We will begin with basic samples with various sentence structures. We will then challenge the BERT-based model with some more difficult samples to explore the system's capacity and limits.

The notebook is an implementation of the Allen Institute for AI BERT-based model. [Reference usage of the Notebook](https://demo.allennlp.org/semantic-role-labeling/MjE4NjI1Ng==)

The BERT-based model is an implementation of [Peng Shi and Jimmy Lin, (2019), ‘Simple BERT Models for Relation Extraction and Semantic Role Labeling’]( https://arxiv.org/abs/1904.05255)

## Setup

In [ ]:
!pip install allennlp==2.1.0 allennlp-models==2.1.0

In [2]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging
import json

In [ ]:
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz")

We also add two functions to display the JSON object SRL BERT returns.

In [5]:
def head(prediction):
  # Iterating through the json to display excerpt of the prediciton
  for i in prediction['verbs']:
    print('Verb:',i['verb'],i['description'])

def full(prediction):
  #print the full prediction
  print(json.dumps(prediction, indent = 1, sort_keys=True))

##Basic samples

Basic samples seem intuitively simple but can be tricky to analyze. Compound
sentences, adjectives, adverbs, and modals are not easy to identify, even for nonexpert humans.

Let's begin with an easy sample for the transformer.

###Sample 1

The first sample is long but relatively easy for the transformer:

In [6]:
prediction = predictor.predict(sentence="Did Bob really think he could prepare a meal for 50 people in only a few hours?")
head(prediction)

Verb: Did [V: Did] Bob really think he could prepare a meal for 50 people in only a few hours ?
Verb: think Did [ARG0: Bob] [ARGM-ADV: really] [V: think] [ARG1: he could prepare a meal for 50 people in only a few hours] ?
Verb: could Did Bob really think he [V: could] [ARG1: prepare a meal for 50 people in only a few hours] ?
Verb: prepare Did Bob really think [ARG0: he] [ARGM-MOD: could] [V: prepare] [ARG1: a meal for 50 people] [ARGM-TMP: in only a few hours] ?


In [7]:
full(prediction)

{
 "verbs": [
  {
   "description": "[V: Did] Bob really think he could prepare a meal for 50 people in only a few hours ?",
   "tags": [
    "B-V",
    "O",
    "O",
    "O",
    "O",
    "O",
    "O",
    "O",
    "O",
    "O",
    "O",
    "O",
    "O",
    "O",
    "O",
    "O",
    "O",
    "O"
   ],
   "verb": "Did"
  },
  {
   "description": "Did [ARG0: Bob] [ARGM-ADV: really] [V: think] [ARG1: he could prepare a meal for 50 people in only a few hours] ?",
   "tags": [
    "O",
    "B-ARG0",
    "B-ARGM-ADV",
    "B-V",
    "B-ARG1",
    "I-ARG1",
    "I-ARG1",
    "I-ARG1",
    "I-ARG1",
    "I-ARG1",
    "I-ARG1",
    "I-ARG1",
    "I-ARG1",
    "I-ARG1",
    "I-ARG1",
    "I-ARG1",
    "I-ARG1",
    "O"
   ],
   "verb": "think"
  },
  {
   "description": "Did Bob really think he [V: could] [ARG1: prepare a meal for 50 people in only a few hours] ?",
   "tags": [
    "O",
    "O",
    "O",
    "O",
    "O",
    "B-V",
    "B-ARG1",
    "I-ARG1",
    "I-ARG1",
    "I-ARG1",
   

###Sample 2

The following sentence seems easy but contains several verbs:

```Mrs. and Mr. Tomaso went to Europe for vacation and visited Paris and first went to visit the Eiffel Tower.```

In [ ]:
!echo '{"sentence": "Mrs. and Mr. Tomaso went to Europe for vacation and visited Paris and first went to visit the Eiffel Tower."}' | \
allennlp predict https://storage.googleapis.com/allennlp-public-models/bert-base-srl-2020.03.24.tar.gz -

input 0:  {"sentence": "Mrs. and Mr. Tomaso went to Europe for vacation and visited Paris and first went to visit the Eiffel Tower."}
prediction:  {"verbs": [{"verb": "went", "description": "[ARG0: Mrs. and Mr. Tomaso] [V: went] [ARG4: to Europe] [ARGM-PRP: for vacation] and visited Paris and first went to visit the Eiffel Tower .", "tags": ["B-ARG0", "I-ARG0", "I-ARG0", "I-ARG0", "B-V", "B-ARG4", "I-ARG4", "B-ARGM-PRP", "I-ARGM-PRP", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"]}, {"verb": "visited", "description": "[ARG0: Mrs. and Mr. Tomaso] went to Europe for vacation and [V: visited] [ARG1: Paris] and first went to visit the Eiffel Tower .", "tags": ["B-ARG0", "I-ARG0", "I-ARG0", "I-ARG0", "O", "O", "O", "O", "O", "O", "B-V", "B-ARG1", "O", "O", "O", "O", "O", "O", "O", "O", "O"]}, {"verb": "went", "description": "[ARG0: Mrs. and Mr. Tomaso] went to Europe for vacation and visited Paris and [ARGM-TMP: first] [V: went] [ARGM-PRP: to visit the Eiffel Tower] .", "tags":

We can interpret the arguments of the verb "went." However, the transformer found that the modifier of the verb was the purpose of the trip.

We can also notice that "went" was correctly associated with "Europe". The
transformer correctly identified the verb "visit" as being related to "Paris".

The transformer could have associated the verb "visited" directly with the "Eiffel Tower". But it didn't. It stood its ground and made the right decision.

The verb "went" was used twice, but the transformer did not fall into the trap. It even found that "first" was a temporal modifier of the verb "went."



###Sample 3

Now we will will make things more difficult for our transformer model. The following sample contains the verb "drink" four times:

```John wanted to drink tea, Mary likes to drink coffee but Karim drank some cool water and Faiza would like to drink tomato juice.```

In [ ]:
!echo '{"sentence": "John wanted to drink tea, Mary likes to drink coffee but Karim drank some cool water and Faiza would like to drink tomato juice."}' | \
allennlp predict https://storage.googleapis.com/allennlp-public-models/bert-base-srl-2020.03.24.tar.gz -

input 0:  {"sentence": "John wanted to drink tea, Mary likes to drink coffee but Karim drank some cool water and Faiza would like to drink tomato juice."}
prediction:  {"verbs": [{"verb": "wanted", "description": "[ARG0: John] [V: wanted] [ARG1: to drink tea] , Mary likes to drink coffee but Karim drank some cool water and Faiza would like to drink tomato juice .", "tags": ["B-ARG0", "B-V", "B-ARG1", "I-ARG1", "I-ARG1", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"]}, {"verb": "drink", "description": "[ARG0: John] wanted to [V: drink] [ARG1: tea] , Mary likes to drink coffee but Karim drank some cool water and Faiza would like to drink tomato juice .", "tags": ["B-ARG0", "O", "O", "B-V", "B-ARG1", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"]}, {"verb": "likes", "description": "John wanted to drink tea , [ARG0: Mary] [V: likes] [ARG1: to drink coffee] but Karim drank som

The first one is perfect. it identifies the verb "wanted" and makes the right
associations.

However, when it identified the verb "drank," it slipped in "and Faiza" as an
argument.

The sentence meant that "Karim drank some cool water." The presence of "and
Faiza" as an argument of "drank" is debatable.

The presence of "some cool water and" is not an argument of like. Only "Faiza" is an argument of "like.". The output is a bit fuzzy.

##Difficult samples

Now, we will run samples that contain problems that the BERT-based
transformer will first solve. We will end with an intractable sample.

Let's start with a complex sample that the BERT-based transformer can analyze.

###Sample 4

It takes us into more tricky SRL territory. The sample separates "Alice" from
the verb "liked," creating a long-term dependency that has to jump over "whose
husband went jogging every Sunday."

The sentence is:

```Alice, whose husband went jogging every Sunday, liked to go to a dancing class in the meantime.```

A human can isolate "Alice" and find the predicate:

```
[Alice, whose husband went jogging every Sunday], liked to go to a dancing
class in the meantime.```

Can the BERT model find the predicate like us?

In [ ]:
!echo '{"sentence": "Alice, whose husband went jogging every Sunday, liked to go to a dancing class in the meantime."}' | \
allennlp predict https://storage.googleapis.com/allennlp-public-models/bert-base-srl-2020.03.24.tar.gz -

input 0:  {"sentence": "Alice, whose husband went jogging every Sunday, liked to go to a dancing class in the meantime."}
prediction:  {"verbs": [{"verb": "went", "description": "Alice , [ARG1: whose husband] [V: went] [ARG2: jogging] [ARGM-TMP: every Sunday] , liked to go to a dancing class in the meantime .", "tags": ["O", "O", "B-ARG1", "I-ARG1", "B-V", "B-ARG2", "B-ARGM-TMP", "I-ARGM-TMP", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"]}, {"verb": "jogging", "description": "Alice , [ARG0: whose husband] went [V: jogging] [ARGM-TMP: every Sunday] , liked to go to a dancing class in the meantime .", "tags": ["O", "O", "B-ARG0", "I-ARG0", "O", "B-V", "B-ARGM-TMP", "I-ARGM-TMP", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"]}, {"verb": "liked", "description": "[ARG0: Alice , whose husband went jogging every Sunday] , [V: liked] [ARG1: to go to a dancing class in the meantime] .", "tags": ["B-ARG0", "I-ARG0", "I-ARG0", "I-ARG0", "I-ARG0", "I-ARG0", "I-ARG0", "I-

Let's focus on the part we are interested in and see if the model finds the predicate. It did! It found the verb "liked" as shown in this excerpt of the raw output.

The transformer explains that:
- The predicate or verb is "went"
- "whose husband" is the argument
- "jogging" is another argument related to "went"
- "every Sunday" is a temporal modifier represented in the raw output as
[ARGM-TMP: every Sunday]

We can see that the verb "jogging" was identified and was related to "whose husband" with the temporal modifier "every Sunday."

The argument describing Alice is a bit long but correct.

We can see that the temporal modifier "in the meantime" was identified as well. It is quite a performance when we think of the simple sequence + verb input the BERTbased model was trained with.

Finally, the transformer identifies the last verb, "dancing," as being related to "class".

###Sample 5

Sample 5 does not repeat a verb several times. However, Sample 5 contains a word
that can have multiple functions and meanings. It goes beyond polysemy since
the word "round" can have both different meanings and grammatical functions.
The word "round" can be a noun, an adjective, an adverb, a transitive verb, or an intransitive verb.

As a transitive or intransitive verb, "round" can mean to attain perfection or
completion. In this sense, "round" can be used with "off."

The following sentence uses "round" in the past tense:

```The bright sun, the blue sky, the warm sand, the palm trees, everything round off.```

Round is used in a sense "to bring to perfection". The best grammatical form would have been "rounded," but the transformer found the right verb, and the sentence sounds rather poetic.

In [ ]:
!echo '{"sentence": "The bright sun, the blue sky, the warm sand, the palm trees, everything round off."}' | \
allennlp predict https://storage.googleapis.com/allennlp-public-models/bert-base-srl-2020.03.24.tar.gz -

input 0:  {"sentence": "The bright sun, the blue sky, the warm sand, the palm trees, everything round off."}
prediction:  {"verbs": [], "words": ["The", "bright", "sun", ",", "the", "blue", "sky", ",", "the", "warm", "sand", ",", "the", "palm", "trees", ",", "everything", "round", "off", "."]}

2021-06-21 06:22:32,844 - INFO - allennlp.models.archival - removing temporary unarchived model dir at /tmp/tmpkqk6_nyt


The output shows no verbs. The transformer did not identify the predicate. In fact, it found no verbs at all.

Since we like our BERT-based transformer, we will be kind to it. Let's change the sentence from the past tense to the present tense.

```The bright sun, the blue sky, the warm sand, the palm trees, everything rounds off.```



In [ ]:
!echo '{"sentence": "The bright sun, the blue sky, the warm sand, the palm trees, everything rounds off."}' | \
allennlp predict https://storage.googleapis.com/allennlp-public-models/bert-base-srl-2020.03.24.tar.gz -

input 0:  {"sentence": "The bright sun, the blue sky, the warm sand, the palm trees, everything rounds off."}
prediction:  {"verbs": [{"verb": "rounds", "description": "[ARG1: The bright sun , the blue sky , the warm sand , the palm trees] , [R-ARG1: everything] [V: rounds] off .", "tags": ["B-ARG1", "I-ARG1", "I-ARG1", "I-ARG1", "I-ARG1", "I-ARG1", "I-ARG1", "I-ARG1", "I-ARG1", "I-ARG1", "I-ARG1", "I-ARG1", "I-ARG1", "I-ARG1", "I-ARG1", "O", "B-R-ARG1", "B-V", "O", "O"]}], "words": ["The", "bright", "sun", ",", "the", "blue", "sky", ",", "the", "warm", "sand", ",", "the", "palm", "trees", ",", "everything", "rounds", "off", "."]}

2021-06-21 06:26:40,005 - INFO - allennlp.models.archival - removing temporary unarchived model dir at /tmp/tmpp1pcwdjm


The raw output shows that the predicate was found.

Our BERT-based transformer did well because the word "round" can be found as
"rounds" in its plural form.

The BERT model initially failed to produce the result we expected. But with a little help from its friends, all ended well for this sample.

###Sample 6

Sample 6 takes a word we often think is just a noun. However, more words than
we suspect can be both nouns and verbs. "To ice" is a verb used in hockey to shoot a "puck" all the way across the rink and beyond the goal line of an opponent. The "puck" is the disk used in hockey.

A hockey coach can start the day by telling a team to train icing pucks. We then can obtain the imperative sentence when the coach yells:

```Now, ice pucks guys!```

Note that "guys" can mean "persons" regardless of their sex.

In [ ]:
!echo '{"sentence": "Now, ice pucks guys!"}' | \
allennlp predict https://storage.googleapis.com/allennlp-public-models/bert-base-srl-2020.03.24.tar.gz -

input 0:  {"sentence": "Now, ice pucks guys!"}
prediction:  {"verbs": [], "words": ["Now", ",", "ice", "pucks", "guys", "!"]}

2021-06-21 06:34:18,257 - INFO - allennlp.models.archival - removing temporary unarchived model dir at /tmp/tmprznv5ksm


Game over! We can see that transformers have made tremendous progress, but there
is still a lot of room for developers to improve the models. Humans are still in the game!